# Data Quality — Overview

## Purpose

Data quality is the foundation of trustworthy analytics and reliable data-driven decision-making. This notebook provides a comprehensive overview of data quality concepts, dimensions, validation strategies, and tools that help ensure your data pipelines produce accurate, consistent, and timely data.

## Key Questions

1. What are the core dimensions of data quality?
2. How do we measure and validate data quality?
3. What tools and frameworks are available for data quality management?
4. How do data contracts and SLAs help maintain data quality?
5. What are best practices for implementing data quality in production pipelines?

---

## 1. Data Quality Dimensions

Data quality is typically evaluated across multiple dimensions. Understanding these dimensions helps identify quality issues and establish meaningful metrics.

### 1.1 Accuracy

**Definition**: The degree to which data correctly represents the real-world entity or event it describes.

| Aspect | Description | Example |
|--------|-------------|--------|
| **Semantic Accuracy** | Data values match real-world truth | Customer age = 35 (actual age is 35) |
| **Syntactic Accuracy** | Data conforms to expected format | Email follows `user@domain.com` pattern |
| **Precision** | Level of detail in measurements | Temperature recorded to 2 decimal places |

```python
# Example: Accuracy validation
def validate_accuracy(df, column, valid_range):
    """Check if values fall within expected range."""
    min_val, max_val = valid_range
    invalid = df[(df[column] < min_val) | (df[column] > max_val)]
    accuracy_rate = 1 - (len(invalid) / len(df))
    return accuracy_rate
```

### 1.2 Completeness

**Definition**: The extent to which all required data is present and no values are missing.

| Type | Description | Measurement |
|------|-------------|------------|
| **Column Completeness** | Percentage of non-null values in a column | `1 - (null_count / total_rows)` |
| **Row Completeness** | Percentage of complete rows (no nulls) | `complete_rows / total_rows` |
| **Schema Completeness** | All expected columns are present | `present_columns / expected_columns` |

```python
# Example: Completeness check
def check_completeness(df, required_columns):
    """Calculate completeness metrics for a DataFrame."""
    results = {}
    for col in required_columns:
        if col in df.columns:
            completeness = df[col].notna().mean()
            results[col] = round(completeness * 100, 2)
        else:
            results[col] = 0.0  # Column missing
    return results
```

### 1.3 Consistency

**Definition**: Data values are uniform and non-contradictory across datasets, systems, and time.

| Consistency Type | Description | Example |
|------------------|-------------|--------|
| **Intra-record** | Values within a single record are consistent | `end_date >= start_date` |
| **Inter-record** | Values across related records match | Order total = sum of line items |
| **Cross-system** | Same entity has same values across systems | Customer name in CRM = name in billing |
| **Temporal** | Historical data remains consistent over time | Yesterday's final figures unchanged |

```python
# Example: Consistency validation
def check_referential_integrity(orders_df, customers_df):
    """Verify all orders reference valid customers."""
    valid_customer_ids = set(customers_df['customer_id'])
    orphan_orders = orders_df[~orders_df['customer_id'].isin(valid_customer_ids)]
    integrity_rate = 1 - (len(orphan_orders) / len(orders_df))
    return integrity_rate, orphan_orders
```

### 1.4 Timeliness

**Definition**: Data is available when needed and reflects the current state of the real world.

| Metric | Description | Calculation |
|--------|-------------|------------|
| **Freshness** | How recent is the data? | `current_time - last_update_time` |
| **Latency** | Time from event to availability | `available_time - event_time` |
| **SLA Compliance** | Data arrives within agreed timeframe | `on_time_deliveries / total_deliveries` |

```python
# Example: Timeliness check
from datetime import datetime, timedelta

def check_freshness(df, timestamp_col, max_age_hours=24):
    """Check if data is within acceptable freshness window."""
    latest_record = df[timestamp_col].max()
    age = datetime.now() - latest_record
    is_fresh = age <= timedelta(hours=max_age_hours)
    return is_fresh, age
```

### Additional Dimensions

| Dimension | Description |
|-----------|------------|
| **Uniqueness** | No duplicate records exist for the same entity |
| **Validity** | Data conforms to defined business rules and constraints |
| **Integrity** | Relationships between data elements are maintained |
| **Conformity** | Data follows standard formats and conventions |

---

## 2. Data Validation Strategies

### 2.1 Validation Approaches

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Data Validation Strategies                        │
├─────────────────┬─────────────────┬─────────────────┬───────────────┤
│   Schema-Based  │  Rule-Based     │  Statistical    │  ML-Based     │
├─────────────────┼─────────────────┼─────────────────┼───────────────┤
│ • Data types    │ • Business      │ • Distribution  │ • Anomaly     │
│ • Constraints   │   logic checks  │   analysis      │   detection   │
│ • Nullability   │ • Regex         │ • Outlier       │ • Pattern     │
│ • Referential   │   patterns      │   detection     │   learning    │
│   integrity     │ • Range checks  │ • Drift         │ • Profiling   │
│                 │                 │   detection     │               │
└─────────────────┴─────────────────┴─────────────────┴───────────────┘
```

### 2.2 Schema Validation

```python
# Using Pydantic for schema validation
from pydantic import BaseModel, validator, EmailStr
from typing import Optional
from datetime import date

class CustomerRecord(BaseModel):
    customer_id: int
    name: str
    email: EmailStr
    registration_date: date
    age: Optional[int] = None
    
    @validator('age')
    def validate_age(cls, v):
        if v is not None and (v < 0 or v > 150):
            raise ValueError('Age must be between 0 and 150')
        return v
```

### 2.3 Rule-Based Validation

```python
# Custom validation rules
class DataQualityRules:
    def __init__(self, df):
        self.df = df
        self.results = []
    
    def check_not_null(self, column):
        """Column should not contain null values."""
        null_count = self.df[column].isna().sum()
        passed = null_count == 0
        self.results.append({
            'rule': f'{column}_not_null',
            'passed': passed,
            'details': f'{null_count} null values found'
        })
        return self
    
    def check_unique(self, column):
        """Column should contain unique values."""
        duplicate_count = self.df[column].duplicated().sum()
        passed = duplicate_count == 0
        self.results.append({
            'rule': f'{column}_unique',
            'passed': passed,
            'details': f'{duplicate_count} duplicates found'
        })
        return self
    
    def check_range(self, column, min_val, max_val):
        """Values should be within specified range."""
        out_of_range = self.df[
            (self.df[column] < min_val) | (self.df[column] > max_val)
        ]
        passed = len(out_of_range) == 0
        self.results.append({
            'rule': f'{column}_in_range',
            'passed': passed,
            'details': f'{len(out_of_range)} values out of range'
        })
        return self
    
    def get_results(self):
        return self.results
```

### 2.4 Statistical Validation

```python
import numpy as np
from scipy import stats

class StatisticalValidator:
    def __init__(self, baseline_df, current_df):
        self.baseline = baseline_df
        self.current = current_df
    
    def detect_distribution_drift(self, column, threshold=0.05):
        """Detect if distribution has shifted using KS test."""
        statistic, p_value = stats.ks_2samp(
            self.baseline[column].dropna(),
            self.current[column].dropna()
        )
        drift_detected = p_value < threshold
        return {
            'column': column,
            'drift_detected': drift_detected,
            'p_value': p_value,
            'ks_statistic': statistic
        }
    
    def detect_outliers(self, column, method='zscore', threshold=3):
        """Identify outliers using z-score method."""
        if method == 'zscore':
            z_scores = np.abs(stats.zscore(self.current[column].dropna()))
            outliers = z_scores > threshold
            return outliers.sum(), outliers.mean()
        elif method == 'iqr':
            Q1 = self.current[column].quantile(0.25)
            Q3 = self.current[column].quantile(0.75)
            IQR = Q3 - Q1
            outliers = (self.current[column] < Q1 - 1.5*IQR) | \
                       (self.current[column] > Q3 + 1.5*IQR)
            return outliers.sum(), outliers.mean()
```

### 2.5 Validation Placement in Pipelines

```
┌──────────┐    ┌──────────┐    ┌───────────┐    ┌──────────┐    ┌──────────┐
│  Source  │───▶│ Ingest   │───▶│ Transform │───▶│  Load    │───▶│  Serve   │
└──────────┘    └──────────┘    └───────────┘    └──────────┘    └──────────┘
                     │               │               │               │
                     ▼               ▼               ▼               ▼
               ┌──────────┐    ┌───────────┐   ┌──────────┐    ┌──────────┐
               │ Schema   │    │ Business  │   │ Integrity│    │ Freshness│
               │ Checks   │    │ Rules     │   │ Checks   │    │ Checks   │
               └──────────┘    └───────────┘   └──────────┘    └──────────┘
```

---

## 3. Great Expectations and Other Tools

### 3.1 Great Expectations

Great Expectations (GX) is the leading open-source data quality framework.

**Core Concepts:**

| Concept | Description |
|---------|------------|
| **Expectation** | A declarative assertion about data (e.g., column is not null) |
| **Expectation Suite** | A collection of expectations for a dataset |
| **Validator** | Validates data against expectation suites |
| **Checkpoint** | Bundles validation runs with actions |
| **Data Docs** | Auto-generated documentation and reports |

```python
# Great Expectations Example
import great_expectations as gx

# Initialize context
context = gx.get_context()

# Connect to data
datasource = context.sources.add_pandas("my_datasource")
data_asset = datasource.add_dataframe_asset(name="orders")

# Build batch request
batch_request = data_asset.build_batch_request(dataframe=orders_df)

# Create expectations
expectation_suite = context.add_expectation_suite("orders_suite")

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="orders_suite"
)

# Add expectations
validator.expect_column_to_exist("order_id")
validator.expect_column_values_to_not_be_null("order_id")
validator.expect_column_values_to_be_unique("order_id")
validator.expect_column_values_to_be_between(
    column="amount",
    min_value=0,
    max_value=100000
)
validator.expect_column_values_to_match_regex(
    column="email",
    regex=r"^[\w\.-]+@[\w\.-]+\.\w+$"
)

# Validate and get results
validation_result = validator.validate()
print(f"Success: {validation_result.success}")
```

### 3.2 Tool Comparison

| Tool | Type | Best For | Key Features |
|------|------|----------|-------------|
| **Great Expectations** | Framework | Comprehensive DQ | Expectations, Data Docs, Profiling |
| **dbt tests** | SQL-based | dbt users | Built-in + custom tests, CI/CD |
| **Soda Core** | CLI/Cloud | SQL warehouses | SodaCL language, monitoring |
| **Pandera** | Python | Pandas/Pyspark | Schema + statistical validation |
| **Deequ** | Spark | Big data | Unit tests for data, anomaly detection |
| **Monte Carlo** | SaaS | Enterprise | ML-powered, observability |
| **Datafold** | SaaS | Data diffing | Column-level lineage, CI testing |

### 3.3 dbt Tests

```yaml
# schema.yml - dbt tests
version: 2

models:
  - name: orders
    columns:
      - name: order_id
        tests:
          - unique
          - not_null
      - name: customer_id
        tests:
          - not_null
          - relationships:
              to: ref('customers')
              field: customer_id
      - name: status
        tests:
          - accepted_values:
              values: ['pending', 'shipped', 'delivered', 'cancelled']
      - name: amount
        tests:
          - dbt_utils.expression_is_true:
              expression: ">= 0"
```

### 3.4 Soda Core

```yaml
# checks.yml - Soda checks
checks for orders:
  - row_count > 0
  - missing_count(order_id) = 0
  - duplicate_count(order_id) = 0
  - invalid_percent(email) < 5%:
      valid regex: "^[\\w\\.-]+@[\\w\\.-]+\\.\\w+$"
  - avg(amount) between 50 and 500
  - freshness(created_at) < 1d
  - schema:
      fail:
        when missing column: [order_id, customer_id, amount]
        when wrong type:
          order_id: integer
          amount: decimal
```

### 3.5 Pandera

```python
import pandera as pa
from pandera import Column, Check, DataFrameSchema

# Define schema with validation rules
order_schema = DataFrameSchema({
    "order_id": Column(int, Check.greater_than(0), unique=True),
    "customer_id": Column(int, Check.greater_than(0)),
    "amount": Column(float, Check.in_range(0, 100000)),
    "status": Column(str, Check.isin(['pending', 'shipped', 'delivered'])),
    "email": Column(str, Check.str_matches(r'^[\w\.-]+@[\w\.-]+\.\w+$')),
    "created_at": Column(pa.DateTime),
})

# Validate DataFrame
validated_df = order_schema.validate(orders_df)

# Use as decorator
@pa.check_input(order_schema)
def process_orders(df):
    # Processing logic here
    return df
```

---

## 4. Data Contracts and SLAs

### 4.1 What is a Data Contract?

A **data contract** is a formal agreement between data producers and consumers that defines:

- Schema and data types
- Semantic meaning of fields
- Quality expectations
- Freshness requirements
- Access patterns
- Ownership and responsibilities

```
┌─────────────────────────────────────────────────────────────────────┐
│                         Data Contract                                │
├─────────────────────────────────────────────────────────────────────┤
│  Producer: Team A                    Consumer: Team B               │
├─────────────────────────────────────────────────────────────────────┤
│  ┌─────────────┐    ┌──────────────┐    ┌─────────────┐            │
│  │   Schema    │    │   Quality    │    │    SLA      │            │
│  │  Definition │    │   Rules      │    │  Agreement  │            │
│  └─────────────┘    └──────────────┘    └─────────────┘            │
├─────────────────────────────────────────────────────────────────────┤
│  Versioning  │  Breaking Change Policy  │  Deprecation Process     │
└─────────────────────────────────────────────────────────────────────┘
```

### 4.2 Data Contract Example (YAML)

```yaml
# data_contract.yaml
dataContractSpecification: 0.9.0
id: urn:datacontract:orders:v1
info:
  title: Orders Data Contract
  version: 1.0.0
  owner: data-platform-team
  contact:
    name: Data Platform Team
    email: data-platform@company.com

servers:
  production:
    type: bigquery
    project: analytics-prod
    dataset: orders

models:
  orders:
    description: Customer order records
    type: table
    fields:
      - name: order_id
        type: string
        required: true
        unique: true
        description: Unique order identifier
      - name: customer_id
        type: string
        required: true
        description: Reference to customer
      - name: amount
        type: decimal
        required: true
        description: Order total in USD
        constraints:
          minimum: 0
      - name: created_at
        type: timestamp
        required: true
        description: Order creation timestamp

quality:
  type: great-expectations
  specification:
    - expectation: expect_table_row_count_to_be_between
      kwargs:
        min_value: 1000
    - expectation: expect_column_values_to_not_be_null
      kwargs:
        column: order_id

sla:
  freshness:
    max_age: PT1H  # ISO 8601 duration: 1 hour
  availability:
    uptime: 99.9%
  latency:
    p95: 500ms
```

### 4.3 Service Level Agreements (SLAs)

| SLA Component | Description | Example |
|---------------|-------------|--------|
| **Freshness** | Maximum age of data | Data updated every hour |
| **Availability** | Uptime percentage | 99.9% availability |
| **Completeness** | Minimum completeness | >99% non-null for required fields |
| **Accuracy** | Error rate threshold | <0.1% data errors |
| **Latency** | Query response time | P95 < 500ms |

### 4.4 Implementing Data Contracts

```python
from dataclasses import dataclass
from datetime import timedelta
from typing import List, Dict

@dataclass
class SLA:
    freshness: timedelta
    completeness_threshold: float
    accuracy_threshold: float
    availability_target: float

@dataclass 
class DataContract:
    name: str
    version: str
    owner: str
    schema: Dict
    quality_rules: List[Dict]
    sla: SLA
    
    def validate(self, df) -> Dict:
        """Validate data against contract."""
        results = {
            'schema_valid': self._validate_schema(df),
            'quality_passed': self._run_quality_checks(df),
            'sla_met': self._check_sla(df)
        }
        results['contract_fulfilled'] = all(results.values())
        return results
    
    def _validate_schema(self, df) -> bool:
        # Schema validation logic
        return True
    
    def _run_quality_checks(self, df) -> bool:
        # Quality rule execution
        return True
    
    def _check_sla(self, df) -> bool:
        # SLA verification
        return True
```

### 4.5 Contract Testing in CI/CD

```yaml
# .github/workflows/data-contract-ci.yml
name: Data Contract Validation

on:
  pull_request:
    paths:
      - 'contracts/**'
      - 'schemas/**'

jobs:
  validate-contracts:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Validate contract syntax
        run: |
          pip install datacontract-cli
          datacontract lint contracts/*.yaml
          
      - name: Check breaking changes
        run: |
          datacontract breaking contracts/orders.yaml --with main
          
      - name: Test with sample data
        run: |
          datacontract test contracts/orders.yaml \
            --examples samples/orders.json
```

---

## 5. Data Quality Architecture

### 5.1 Centralized vs Decentralized Approach

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Centralized DQ                                    │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐                       │
│  │ Team A   │    │ Team B   │    │ Team C   │                       │
│  └────┬─────┘    └────┬─────┘    └────┬─────┘                       │
│       │               │               │                              │
│       └───────────────┼───────────────┘                              │
│                       ▼                                              │
│              ┌──────────────────┐                                    │
│              │  Central DQ Team │                                    │
│              └──────────────────┘                                    │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│                    Decentralized (Mesh) DQ                          │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐              │
│  │   Domain A   │  │   Domain B   │  │   Domain C   │              │
│  │ ┌──────────┐ │  │ ┌──────────┐ │  │ ┌──────────┐ │              │
│  │ │ DQ Rules │ │  │ │ DQ Rules │ │  │ │ DQ Rules │ │              │
│  │ └──────────┘ │  │ └──────────┘ │  │ └──────────┘ │              │
│  └──────────────┘  └──────────────┘  └──────────────┘              │
│                         │                                           │
│                         ▼                                           │
│              ┌──────────────────────┐                               │
│              │  Shared DQ Platform  │                               │
│              │  (Standards + Tools) │                               │
│              └──────────────────────┘                               │
└─────────────────────────────────────────────────────────────────────┘
```

### 5.2 Data Quality Monitoring Dashboard

Key metrics to track:

| Metric Category | Metrics |
|-----------------|--------|
| **Coverage** | % tables with DQ checks, % columns validated |
| **Pass Rate** | % checks passing, trend over time |
| **Incidents** | DQ failures by severity, MTTR |
| **SLA Compliance** | Freshness SLA met, availability % |
| **Data Volume** | Row counts, anomaly detection |

---

## 6. Best Practices

### 6.1 Implementation Guidelines

| Practice | Description |
|----------|------------|
| **Shift Left** | Validate early in the pipeline, close to the source |
| **Automate** | Integrate DQ checks into CI/CD and orchestration |
| **Alert Smartly** | Set thresholds to avoid alert fatigue |
| **Document** | Maintain living documentation with data contracts |
| **Measure ROI** | Track cost of bad data vs investment in DQ |
| **Iterate** | Start simple, add complexity based on needs |

### 6.2 Common Anti-Patterns

| Anti-Pattern | Why It's Bad | Better Approach |
|--------------|--------------|----------------|
| Validating only at the end | Late detection, expensive fixes | Validate at each stage |
| Too many alerts | Alert fatigue, ignored warnings | Prioritize critical checks |
| No baseline metrics | Can't detect drift | Profile data regularly |
| Manual quality checks | Not scalable, error-prone | Automate everything |
| Siloed DQ ownership | Gaps between teams | Shared responsibility model |

---

## 🎯 Takeaway

| Concept | Key Points |
|---------|------------|
| **Dimensions** | Focus on accuracy, completeness, consistency, and timeliness as core quality pillars |
| **Validation** | Layer schema, rule-based, and statistical validation throughout your pipelines |
| **Tools** | Great Expectations, dbt tests, Soda, and Pandera provide robust validation frameworks |
| **Contracts** | Data contracts formalize agreements between producers and consumers |
| **SLAs** | Define measurable quality targets for freshness, availability, and accuracy |
| **Architecture** | Balance centralized standards with decentralized domain ownership |

### Quick Reference Commands

```bash
# Great Expectations
pip install great_expectations
great_expectations init
great_expectations checkpoint run my_checkpoint

# Soda Core
pip install soda-core
soda scan -d my_datasource -c configuration.yml checks.yml

# dbt tests
dbt test --select model_name
dbt test --select tag:data_quality
```

### Further Reading

- [Great Expectations Documentation](https://docs.greatexpectations.io/)
- [Soda Core Documentation](https://docs.soda.io/)
- [Data Contracts Specification](https://datacontract.com/)
- [dbt Testing Documentation](https://docs.getdbt.com/docs/build/tests)